In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import math
import re
from collections import Counter
from tqdm import tqdm_notebook as tqdm
import numpy as np

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [3]:
from keras.utils import multi_gpu_model
from keras_bert import load_vocabulary, load_trained_model_from_checkpoint, Tokenizer, get_checkpoint_paths
from keras.layers import *
from keras.models import Model
import keras.backend as K
from keras.optimizers import Adam
from keras.callbacks import Callback, ModelCheckpoint
from keras.utils.data_utils import Sequence
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.
W0807 07:50:25.567746 140191956285248 deprecation_wrapper.py:118] From /usr/local/lib/python3.6/dist-packages/tensorflow_estimator/python/estimator/api/_v1/estimator/__init__.py:10: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.



In [4]:
from nl2sql.utils import read_data, read_tables, SQL, MultiSentenceTokenizer, Query
from nl2sql.utils import SQL

In [5]:
USE_MULTI_GPUS = True

train_table_file = '../data/train/train.tables.json'
train_data_file = '../data/train/train.json'

val_table_file = '../data/val/val.tables.json'
val_data_file = '../data/val/val.json'

test_table_file = '../data/test/test.tables.json'
test_data_file = '../data/test/test.json'

bert_model_path = '../model/chinese_wwm_L-12_H-768_A-12'
paths = get_checkpoint_paths(bert_model_path)

In [6]:
train_tables = read_tables(train_table_file)
train_data = read_data(train_data_file, train_tables)

val_tables = read_tables(val_table_file)
val_data = read_data(val_data_file, val_tables)

In [7]:
token_dict = load_vocabulary(paths.vocab)

In [8]:
class QueryTokenizer(MultiSentenceTokenizer):
    col_type_token_dict = {'text': '[unused11]', 'real': '[unused12]'}
    
    def tokenize(self, query: Query):
        question_tokens = [self._token_cls] + self._tokenize(query.question.text)
        header_tokens = []

        for col_name, col_type in query.table.header:
            col_type_token = self.col_type_token_dict[col_type]
            col_name_tokens = self._tokenize(col_name)
            header_tokens.append([col_type_token] + col_name_tokens)
        all_tokens = [question_tokens] + header_tokens
        return self._pack(*all_tokens)
    
    def encode(self, query:Query):
        tokens, tokens_lens = self.tokenize(query)
        token_ids = self._convert_tokens_to_ids(tokens)
        segment_ids = [0] * len(token_ids)
        header_indices = np.cumsum(tokens_lens)
        return token_ids, segment_ids, header_indices[:-1]

In [9]:
query_tokenizer = QueryTokenizer(token_dict)

In [10]:
sample_query = train_data[0]
print('QueryTokenizer')
print('Input Question: {}'.format(sample_query.question))
print('Input Header: {}'.format(sample_query.table.header))
print('Output Tokens: {}'.format(' '.join(query_tokenizer.tokenize(sample_query)[0])))
print('Output Encoded: {}'.format(query_tokenizer.encode(sample_query)))

QueryTokenizer
Input Question: 二零一九年第四周大黄蜂和密室逃生这两部影片的票房总占比是多少呀
Input Header: 影片名称(text) | 周票房（万）(real) | 票房占比（%）(real) | 场均人次(real)
Output Tokens: [CLS] 二 零 一 九 年 第 四 周 大 黄 蜂 和 密 室 逃 生 这 两 部 影 片 的 票 房 总 占 比 是 多 少 呀 [SEP] [unused11] 影 片 名 称 [SEP] [unused12] 周 票 房 （ 万 ） [SEP] [unused12] 票 房 占 比 （ % ） [SEP] [unused12] 场 均 人 次 [SEP]
Output Encoded: ([101, 753, 7439, 671, 736, 2399, 5018, 1724, 1453, 1920, 7942, 6044, 1469, 2166, 2147, 6845, 4495, 6821, 697, 6956, 2512, 4275, 4638, 4873, 2791, 2600, 1304, 3683, 3221, 1914, 2208, 1435, 102, 11, 2512, 4275, 1399, 4917, 102, 12, 1453, 4873, 2791, 8020, 674, 8021, 102, 12, 4873, 2791, 1304, 3683, 8020, 110, 8021, 102, 12, 1767, 1772, 782, 3613, 102], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], array([33, 39, 47, 56]))


In [11]:
class SqlLabelEncoder:
    
    def encode(self, sql: SQL, num_cols):
        cond_conn_op_label = sql.cond_conn_op
        
        sel_agg_label = np.ones(num_cols, dtype='int32') * len(SQL.agg_sql_dict)
        for col_id, agg_op in zip(sql.sel, sql.agg):
            if col_id < num_cols:
                sel_agg_label[col_id] = agg_op
            
        cond_op_label = np.ones(num_cols, dtype='int32') * len(SQL.op_sql_dict)
        for col_id, cond_op, _ in sql.conds:
            if col_id < num_cols:
                cond_op_label[col_id] = cond_op
            
        return cond_conn_op_label, sel_agg_label, cond_op_label
    
    def decode(self, cond_conn_op_label, sel_agg_label, cond_op_label):
        cond_conn_op = int(cond_conn_op_label)
        sel, agg, conds = [], [], []

        for col_id, (agg_op, cond_op) in enumerate(zip(sel_agg_label, cond_op_label)):
            if agg_op < len(SQL.agg_sql_dict):
                sel.append(col_id)
                agg.append(int(agg_op))
            if cond_op < len(SQL.op_sql_dict):
                conds.append([col_id, int(cond_op)])
        return {
            'sel': sel,
            'agg': agg,
            'cond_conn_op': cond_conn_op,
            'conds': conds
        }

In [12]:
label_encoder = SqlLabelEncoder()

In [13]:
label_encoder.decode(*label_encoder.encode(sample_query.sql, 5))

{'sel': [2], 'agg': [5], 'cond_conn_op': 2, 'conds': [[0, 2]]}

In [14]:
class DataSequence(Sequence):
    def __init__(self, data, tokenizer, label_encoder, is_train=True, max_len=160, batch_size=32, shuffle=True, global_indices=None):
        self.data = data
        self.batch_size = batch_size
        self.tokenizer = tokenizer
        self.label_encoder = label_encoder
        self.shuffle = shuffle
        self.is_train = is_train
        self.max_len = max_len
        if global_indices is None:
            self._global_indices = np.arange(len(data))
        else:
            self._global_indices = global_indices
        if shuffle:
            np.random.shuffle(self._global_indices)
    
    def _pad_sequences(self, seqs, max_len=None):
        padded = pad_sequences(seqs, maxlen=None, padding='post')
        if max_len is not None:
            padded = padded[:, :max_len]
        return padded
    
    def __getitem__(self, batch_id):
        batch_data_indices = \
            self._global_indices[batch_id * self.batch_size: (batch_id + 1) * self.batch_size]
        batch_data = [self.data[i] for i in batch_data_indices]
        
        TOKEN_IDS, SEGMENT_IDS = [], []
        HEADER_IDS, HEADER_MASK = [], []
        COND_CONN_OP = []
        SEL_AGG = []
        COND_OP = []
        
        for query in batch_data:
            question = query.question.text
            table = query.table
            
            token_ids, segment_ids, header_ids = self.tokenizer.encode(query)
            header_ids = [hid for hid in header_ids if hid < self.max_len]
            header_mask = [1] * len(header_ids)
            
            TOKEN_IDS.append(token_ids)
            SEGMENT_IDS.append(segment_ids)
            HEADER_IDS.append(header_ids)
            HEADER_MASK.append(header_mask)
            
            if not self.is_train:
                continue
            sql = query.sql
            
            cond_conn_op, sel_agg, cond_op = self.label_encoder.encode(sql, num_cols=len(header_ids))
            
            COND_CONN_OP.append(cond_conn_op)
            SEL_AGG.append(sel_agg)
            COND_OP.append(cond_op)
            
        TOKEN_IDS = self._pad_sequences(TOKEN_IDS, max_len=self.max_len)
        SEGMENT_IDS = self._pad_sequences(SEGMENT_IDS, max_len=self.max_len)
        HEADER_IDS = self._pad_sequences(HEADER_IDS)
        HEADER_MASK = self._pad_sequences(HEADER_MASK)
        
        inputs = {
            'input_token_ids': TOKEN_IDS,
            'input_segment_ids': SEGMENT_IDS,
            'input_header_ids': HEADER_IDS,
            'input_header_mask': HEADER_MASK
        }
        
        if self.is_train:
            SEL_AGG = self._pad_sequences(SEL_AGG)
            SEL_AGG = np.expand_dims(SEL_AGG, axis=-1)
            COND_CONN_OP = np.expand_dims(COND_CONN_OP, axis=-1)
            COND_OP = self._pad_sequences(COND_OP)
            COND_OP = np.expand_dims(COND_OP, axis=-1)

            outputs = {
                'output_sel_agg': SEL_AGG,
                'output_cond_conn_op': COND_CONN_OP,
                'output_cond_op': COND_OP
            }
            return inputs, outputs
        else:
            return inputs
    
    def __len__(self):
        return math.ceil(len(self.data) / self.batch_size)
    
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self._global_indices)

In [15]:
train_seq = DataSequence(train_data, query_tokenizer, label_encoder, shuffle=False, max_len=160, batch_size=2)

In [16]:
sample_inputs, sample_outputs = train_seq[0]
for name, data in sample_inputs.items():
    print('{} : {}'.format(name, data.shape))
#     print(data)
    
for name, data in sample_outputs.items():
    print('{} : {}'.format(name, data.shape))
#     print(data)

input_token_ids : (2, 63)
input_segment_ids : (2, 63)
input_header_ids : (2, 4)
input_header_mask : (2, 4)
output_sel_agg : (2, 4, 1)
output_cond_conn_op : (2, 1)
output_cond_op : (2, 4, 1)


In [17]:
learning_rate = 5e-5
num_sel_agg = len(SQL.agg_sql_dict) + 1
num_cond_op = len(SQL.op_sql_dict) + 1
num_cond_conn_op = len(SQL.conn_sql_dict)

In [18]:
bert_model = load_trained_model_from_checkpoint(paths.config, paths.checkpoint, seq_len=None)
for l in bert_model.layers:
    l.trainable = True

W0807 07:50:29.255158 140191956285248 deprecation_wrapper.py:118] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0807 07:50:29.260613 140191956285248 deprecation_wrapper.py:118] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4140: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0807 07:50:29.308119 140191956285248 deprecation_wrapper.py:118] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:131: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0807 07:50:29.308856 140191956285248 deprecation_wrapper.py:118] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0807 07:50:29.316616

In [19]:
def seq_gather(x):
    seq, idxs = x
    idxs = K.cast(idxs, 'int32')
    return K.tf.batch_gather(seq, idxs)

In [20]:
inp_token_ids = Input(shape=(None,), name='input_token_ids', dtype='int32')
inp_segment_ids = Input(shape=(None,), name='input_segment_ids', dtype='int32')
inp_header_ids = Input(shape=(None,), name='input_header_ids', dtype='int32')
inp_header_mask = Input(shape=(None, ), name='input_header_mask')

x = bert_model([inp_token_ids, inp_segment_ids]) # (None, seq_len, 768)

# predict cond_conn_op
x_for_cond_conn_op = Lambda(lambda x: x[:, 0])(x) # (None, 768)
p_cond_conn_op = Dense(num_cond_conn_op, activation='softmax', name='output_cond_conn_op')(x_for_cond_conn_op)

# predict sel_agg
x_for_header = Lambda(seq_gather, name='header_seq_gather')([x, inp_header_ids]) # (None, h_len, 768)
header_mask = Lambda(lambda x: K.expand_dims(x, axis=-1))(inp_header_mask) # (None, h_len, 1)
x_for_header = Multiply()([x_for_header, header_mask])
x_for_header = Masking()(x_for_header)

p_sel_agg = Dense(num_sel_agg, activation='softmax', name='output_sel_agg')(x_for_header)
p_cond_op = Dense(num_cond_op, activation='softmax', name='output_cond_op')(x_for_header)

model = Model(
    [inp_token_ids, inp_segment_ids, inp_header_ids, inp_header_mask],
    [p_cond_conn_op, p_sel_agg, p_cond_op]
)

W0807 07:50:57.445908 140191956285248 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/util/dispatch.py:180: batch_gather (from tensorflow.python.ops.array_ops) is deprecated and will be removed after 2017-10-25.
Instructions for updating:
`tf.batch_gather` is deprecated, please use `tf.gather` with `batch_dims=-1` instead.


In [21]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_token_ids (InputLayer)    (None, None)         0                                            
__________________________________________________________________________________________________
input_segment_ids (InputLayer)  (None, None)         0                                            
__________________________________________________________________________________________________
model_2 (Model)                 (None, None, 768)    101677056   input_token_ids[0][0]            
                                                                 input_segment_ids[0][0]          
__________________________________________________________________________________________________
input_header_ids (InputLayer)   (None, None)         0                                            
__________

In [27]:
USE_MULTI_GPUS = True
if USE_MULTI_GPUS:
    print('using multi-gpus')
    model = multi_gpu_model(model, gpus=2)
    

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=Adam(learning_rate),
    metrics={
        'output_cond_conn_op': 'accuracy'
    }
)

using multi-gpus


W0807 07:52:17.995884 140191956285248 deprecation_wrapper.py:118] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [34]:
train_dataseq = DataSequence(
    data=train_data,
    tokenizer=query_tokenizer,
    label_encoder=label_encoder,
    is_train=True, 
    max_len=160, 
    batch_size=2
)

val_dataseq = DataSequence(
    data=val_data, 
    tokenizer=query_tokenizer,
    label_encoder=label_encoder,
    is_train=True, 
    max_len=160, 
    shuffle=False,
    batch_size=32
)

In [24]:
def outputs_to_sqls(preds_cond_conn_op, preds_sel_agg, preds_cond_op, header_lens, label_encoder):
    preds_cond_conn_op = np.argmax(preds_cond_conn_op, axis=-1)
    preds_sel_agg = np.argmax(preds_sel_agg, axis=-1)
    preds_cond_op = np.argmax(preds_cond_op, axis=-1)

    sqls = []
    
    for cond_conn_op, sel_agg, cond_op, header_len in zip(preds_cond_conn_op, 
                                                          preds_sel_agg, 
                                                          preds_cond_op, 
                                                          header_lens):
        sql = label_encoder.decode(cond_conn_op, sel_agg, cond_op)
        sql['conds'] = [cond for cond in sql['conds'] if cond[0] < header_len]
        sel = []
        agg = []
        for col_id, agg_type in zip(sql['sel'], sql['agg']):
            if col_id < header_len:
                sel.append(col_id)
                agg.append(agg_type)
        sql['sel'] = sel
        sql['agg'] = agg
        sqls.append(sql)
    return sqls

class EvaluateCallback(Callback):
    def __init__(self, val_dataseq):
        self.val_dataseq = val_dataseq
    
    def on_epoch_end(self, epoch, logs=None):

        is_train = self.val_dataseq.is_train
        self.val_dataseq.is_train = False
        pred_sqls = []
        for batch_data in self.val_dataseq:
            header_lens = np.sum(batch_data['input_header_mask'], axis=-1)
            preds_cond_conn_op, preds_sel_agg, preds_cond_op = self.model.predict_on_batch(batch_data)
            sqls = outputs_to_sqls(preds_cond_conn_op, preds_sel_agg, preds_cond_op, 
                                   header_lens, val_dataseq.label_encoder)
            pred_sqls += sqls
        conn_correct = 0
        agg_correct = 0
        conds_correct = 0
        conds_col_id_correct = 0
        all_correct = 0
        num_queries = len(self.val_dataseq.data)
        
        true_sqls = [query.sql for query in self.val_dataseq.data]
        for pred_sql, true_sql in zip(pred_sqls, true_sqls):
            n_correct = 0
            if pred_sql['cond_conn_op'] == true_sql.cond_conn_op:
                conn_correct += 1
                n_correct += 1
            
            pred_aggs = set(zip(pred_sql['sel'], pred_sql['agg']))
            true_aggs = set(zip(true_sql.sel, true_sql.agg))
            if pred_aggs == true_aggs:
                agg_correct += 1
                n_correct += 1

            pred_conds = set([(cond[0], cond[1]) for cond in pred_sql['conds']])
            true_conds = set([(cond[0], cond[1]) for cond in true_sql.conds])

            if pred_conds == true_conds:
                conds_correct += 1
                n_correct += 1
   
            pred_conds_col_ids = set([cond[0] for cond in pred_sql['conds']])
            true_conds_col_ids = set([cond[0] for cond in true_sql['conds']])
            if pred_conds_col_ids == true_conds_col_ids:
                conds_col_id_correct += 1
            
            if n_correct == 3:
                all_correct += 1

        print('conn_acc: {}'.format(conn_correct / num_queries))
        print('agg_acc: {}'.format(agg_correct / num_queries))
        print('conds_acc: {}'.format(conds_correct / num_queries))
        print('conds_col_id_acc: {}'.format(conds_col_id_correct / num_queries))
        print('total_acc: {}'.format(all_correct / num_queries))
        logs['val_tot_acc'] = all_correct / num_queries
        self.val_dataseq.is_train = is_train

In [25]:
class LearningRateScheduler(Callback):
    def __init__(self, init_lr, min_lr):
        self.passed = 0
        self.init_lr = init_lr
        self.min_lr = min_lr
        
    def on_batch_begin(self, batch, logs=None):
        if self.passed < self.params['steps']:
            lr = (self.passed + 1) / self.params['steps'] * self.init_lr
            K.set_value(self.model.optimizer.lr, lr)
            self.passed += 1
        elif self.params['steps'] <= self.passed < self.params['steps'] * 2:
            lr = (2 - (self.passed + 1) / self.params['steps']) * (self.init_lr - self.min_lr)
            lr += self.min_lr
            K.set_value(self.model.optimizer.lr, lr)
            self.passed += 1

In [26]:
callbacks = [
    LearningRateScheduler(learning_rate, min_lr=1e-5),
    EvaluateCallback(val_dataseq),
    ModelCheckpoint("output/bert_all.{epoch:02d}-{val_tot_acc:.3f}.hd5", monitor='val_tot_acc', mode='max', save_best_only=True)
]

In [31]:
model.fit_generator(train_dataseq, epochs=13, callbacks=callbacks)

Epoch 11/13
649/649 [==============================] - 418s 643ms/step - loss: 0.0178 - output_cond_conn_op_loss: 0.0049 - output_sel_agg_loss: 0.0056 - output_cond_op_loss: 0.0074 - output_cond_conn_op_acc: 0.9986
conn_acc: 0.95950864422202
agg_acc: 0.9499545040946314
conds_acc: 0.8971792538671519
conds_col_id_acc: 0.9069608735213831
total_acc: 0.8416742493175614
Epoch 12/13
649/649 [==============================] - 448s 690ms/step - loss: 0.0157 - output_cond_conn_op_loss: 0.0035 - output_sel_agg_loss: 0.0055 - output_cond_op_loss: 0.0067 - output_cond_conn_op_acc: 0.9991
conn_acc: 0.9638307552320291
agg_acc: 0.9488171064604186
conds_acc: 0.905368516833485
conds_col_id_acc: 0.9156050955414012
total_acc: 0.8519108280254777
Epoch 13/13
649/649 [==============================] - 455s 702ms/step - loss: 0.0152 - output_cond_conn_op_loss: 0.0048 - output_sel_agg_loss: 0.0045 - output_cond_op_loss: 0.0059 - output_cond_conn_op_acc: 0.9982
conn_acc: 0.9597361237488626
agg_acc: 0.9483621474

In [23]:
model.load_weights('../model/bert_all.12-0.852.hd5')

In [36]:
val_dataseq.is_train = False
pred_sqls = []

for batch_data in tqdm(val_dataseq):
    header_lens = np.sum(batch_data['input_header_mask'], axis=-1)
    preds_cond_conn_op, preds_sel_agg, preds_cond_op = model.predict_on_batch(batch_data)
    sqls = outputs_to_sqls(preds_cond_conn_op, preds_sel_agg, preds_cond_op, 
                           header_lens, val_dataseq.label_encoder)
    pred_sqls += sqls

In [40]:
import json

In [44]:
with open('./output/bert_noval_outputs_190722.json', 'w') as f:
    for sql in pred_sqls:
        f.write(json.dumps(sql, ensure_ascii=True) + '\n')

make test prediction

In [50]:
test_tables = read_tables(test_table_file)
test_data = read_data(test_data_file, test_tables)

test_dataseq = DataSequence(
    data=test_data, 
    tokenizer=query_tokenizer,
    label_encoder=label_encoder,
    is_train=False, 
    max_len=160, 
    shuffle=False,
    batch_size=64
)

In [51]:
pred_sqls = []

for batch_data in tqdm(test_dataseq):
    header_lens = np.sum(batch_data['input_header_mask'], axis=-1)
    preds_cond_conn_op, preds_sel_agg, preds_cond_op = model.predict_on_batch(batch_data)
    sqls = outputs_to_sqls(preds_cond_conn_op, preds_sel_agg, preds_cond_op, 
                           header_lens, test_dataseq.label_encoder)
    pred_sqls += sqls

In [52]:
len(pred_sqls)

4086

In [53]:
with open('./output/test_bert_noval_outputs_190722.json', 'w') as f:
    for sql in pred_sqls:
        f.write(json.dumps(sql, ensure_ascii=True) + '\n')